<a href="https://www.kaggle.com/code/vigneshselvaraj2512/scholar-guardian-multi-agent-study-coach-for-stru?scriptVersionId=283154346" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import json
from pathlib import Path
from datetime import datetime

# File where we store simple long-term memory for users
MEMORY_FILE = Path("student_memory.json")


class MemoryStore:
    def __init__(self, path=MEMORY_FILE):
        self.path = Path(path)
        if not self.path.exists():
            self._write({"users": {}})

    def _read(self):
        try:
            with self.path.open("r", encoding="utf-8") as f:
                return json.load(f)
        except json.JSONDecodeError:
            return {"users": {}}

    def _write(self, data):
        with self.path.open("w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)

    def get_user(self, user_id):
        data = self._read()
        return data["users"].get(
            user_id,
            {
                "level": "beginner",
                "topics_studied": [],
                "sessions": []
            },
        )

    def save_user(self, user_id, user_data):
        data = self._read()
        data["users"][user_id] = user_data
        self._write(data)


class TutorAgent:
    def explain_concept(self, topic, level="beginner"):
        topic_lower = topic.lower()
        if "value iteration" in topic_lower:
            return (
                "Value Iteration (beginner friendly):\n"
                "1. Imagine a grid of states. Each state has a 'value' = how good it is to be there.\n"
                "2. We repeatedly update each state's value using the Bellman equation:\n"
                "   V(s) = max_a [ R(s,a) + γ * Σ P(s'|s,a) * V(s') ].\n"
                "3. After many updates, values stop changing much → we have near-optimal values.\n"
                "4. Then the best action in each state is the one with the highest expected value.\n"
                "\nSimple Python-style pseudocode:\n"
                "values = [0]*num_states\n"
                "for iteration in range(100):\n"
                "    new_values = values.copy()\n"
                "    for s in states:\n"
                "        new_values[s] = max_over_actions(...Bellman update...)\n"
                "    values = new_values\n"
            )
        else:
            return (
                f"Here is a simple explanation of '{topic}':\n"
                "- I treat this as a study concept the student wants to understand.\n"
                "- In a real system, this message would be generated by an LLM.\n"
                "- You can plug in your OpenAI agent call here to get richer explanations."
            )

    def create_study_plan(self, goal, days=7):
        return [
            f"Day {i+1}: Focus on part {i+1} of your goal → '{goal}'. "
            "Review notes, watch one tutorial, and write a short summary."
            for i in range(days)
        ]


class QuizAgent:
    def generate_quiz(self, topic):
        return [
            {
                "question": f"In one sentence, what problem does '{topic}' solve?",
                "answer_hint": "Think about why we would use this method at all.",
            },
            {
                "question": f"Name one real-world use case of '{topic}'.",
                "answer_hint": "Consider an example from games, robotics, or recommendation systems.",
            },
        ]


class ProgressAgent:
    def __init__(self, memory: MemoryStore):
        self.memory = memory

    def log_session(self, user_id, topic, notes):
        user = self.memory.get_user(user_id)
        if topic not in user["topics_studied"]:
            user["topics_studied"].append(topic)
        user["sessions"].append(
            {
                "topic": topic,
                "notes": notes,
                "timestamp": datetime.utcnow().isoformat(),
            }
        )
        self.memory.save_user(user_id, user)

    def get_summary(self, user_id):
        user = self.memory.get_user(user_id)
        topics = ", ".join(user["topics_studied"]) or "No topics yet"
        sessions_count = len(user["sessions"])
        return (
            f"Progress summary for {user_id}:\n"
            f"- Level: {user['level']}\n"
            f"- Topics studied: {topics}\n"
            f"- Study sessions logged: {sessions_count}"
        )


class OrchestratorAgent:
    """
    OrchestratorAgent acts like a simple 'brain':
    - Classifies user intent
    - Routes to TutorAgent, QuizAgent, or ProgressAgent
    """

    def __init__(self, memory: MemoryStore):
        self.memory = memory
        self.tutor = TutorAgent()
        self.quiz = QuizAgent()
        self.progress = ProgressAgent(memory)

    def classify_intent(self, message: str) -> str:
        msg = message.lower()
        if any(k in msg for k in ["plan", "schedule", "7 days", "study"]):
            return "STUDY_PLAN"
        if any(k in msg for k in ["quiz", "test", "questions"]):
            return "QUIZ"
        if any(k in msg for k in ["progress", "summary", "what have i done"]):
            return "PROGRESS"
        return "EXPLAIN"

    def handle_message(self, user_id: str, message: str) -> str:
        intent = self.classify_intent(message)

        if intent == "STUDY_PLAN":
            plan = self.tutor.create_study_plan(message, days=7)
            self.progress.log_session(user_id, "Study plan", message)
            return "Here is your 7-day study plan:\n" + "\n".join(plan)

        elif intent == "QUIZ":
            topic = (
                message.replace("quiz", "").replace("Quiz", "").strip()
                or "the topic you last studied"
            )
            questions = self.quiz.generate_quiz(topic)
            self.progress.log_session(
                user_id, f"Quiz on {topic}", "User requested a quiz"
            )
            out_lines = ["Here is a short reflective quiz:"]
            for i, q in enumerate(questions, start=1):
                out_lines.append(
                    f"{i}. {q['question']} (Hint: {q['answer_hint']})"
                )
            return "\n".join(out_lines)

        elif intent == "PROGRESS":
            return self.progress.get_summary(user_id)

        else:  # EXPLAIN
            topic = (
                message.replace("explain", "")
                .replace("Explain", "")
                .strip()
                or message
            )
            explanation = self.tutor.explain_concept(topic)
            self.progress.log_session(
                user_id, topic, "Concept explanation requested"
            )
            return explanation


def demo():
    memory = MemoryStore()
    orchestrator = OrchestratorAgent(memory)

    user_id = "demo_student"
    print("ScholarGuardian – Multi-Agent Study Coach Demo")
    print("Type 'quit' to exit.")
    while True:
        msg = input("\nYou: ")
        if msg.strip().lower() in {"quit", "exit"}:
            break
        response = orchestrator.handle_message(user_id, msg)
        print("\nAgent:\n" + response)


if __name__ == "__main__":
    demo()


ScholarGuardian – Multi-Agent Study Coach Demo
Type 'quit' to exit.



You:  Explain value iteration



Agent:
Value Iteration (beginner friendly):
1. Imagine a grid of states. Each state has a 'value' = how good it is to be there.
2. We repeatedly update each state's value using the Bellman equation:
   V(s) = max_a [ R(s,a) + γ * Σ P(s'|s,a) * V(s') ].
3. After many updates, values stop changing much → we have near-optimal values.
4. Then the best action in each state is the one with the highest expected value.

Simple Python-style pseudocode:
values = [0]*num_states
for iteration in range(100):
    new_values = values.copy()
    for s in states:
        new_values[s] = max_over_actions(...Bellman update...)
    values = new_values




You:  Make a study plan for reinforcement learning



Agent:
Here is your 7-day study plan:
Day 1: Focus on part 1 of your goal → 'Make a study plan for reinforcement learning'. Review notes, watch one tutorial, and write a short summary.
Day 2: Focus on part 2 of your goal → 'Make a study plan for reinforcement learning'. Review notes, watch one tutorial, and write a short summary.
Day 3: Focus on part 3 of your goal → 'Make a study plan for reinforcement learning'. Review notes, watch one tutorial, and write a short summary.
Day 4: Focus on part 4 of your goal → 'Make a study plan for reinforcement learning'. Review notes, watch one tutorial, and write a short summary.
Day 5: Focus on part 5 of your goal → 'Make a study plan for reinforcement learning'. Review notes, watch one tutorial, and write a short summary.
Day 6: Focus on part 6 of your goal → 'Make a study plan for reinforcement learning'. Review notes, watch one tutorial, and write a short summary.
Day 7: Focus on part 7 of your goal → 'Make a study plan for reinforcement lear